<a href="https://colab.research.google.com/github/Marlburo/colab_nb/blob/Keras/colab_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt


In [3]:
#housing_test = pd.read_csv("/content/sample_data/california_housing_test.csv")
#housing_train = pd.read_csv("/content/sample_data/california_housing_train.csv")
mnist_test = pd.read_csv("/content/sample_data/mnist_test.csv")
mnist_train = pd.read_csv("/content/sample_data/mnist_train_small.csv")

In [4]:
#housing_test.describe()
#mnist_test
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name = 'predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()#mnist_train, mnist_test

#preprocess the data
x_train = x_train.reshape(60000, 784).astype("float32")/255
x_test = x_test.reshape(10000, 784).astype("float32")/255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

#reserve 10000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    #loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    #list of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

In [7]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=4,
    #We pass some validation for
    #monitoring validation loss and metrics
    #at the end of each epoch
    validation_data=(x_val, y_val),
)

Fit model on training data
Epoch 1/4
782/782 [==============================] - 3s 3ms/step - loss: 0.3432 - sparse_categorical_accuracy: 0.9017 - val_loss: 0.1846 - val_sparse_categorical_accuracy: 0.9471
Epoch 2/4
782/782 [==============================] - 2s 2ms/step - loss: 0.1566 - sparse_categorical_accuracy: 0.9538 - val_loss: 0.1270 - val_sparse_categorical_accuracy: 0.9620
Epoch 3/4
782/782 [==============================] - 2s 2ms/step - loss: 0.1141 - sparse_categorical_accuracy: 0.9660 - val_loss: 0.1079 - val_sparse_categorical_accuracy: 0.9687
Epoch 4/4
782/782 [==============================] - 2s 2ms/step - loss: 0.0913 - sparse_categorical_accuracy: 0.9724 - val_loss: 0.0955 - val_sparse_categorical_accuracy: 0.9726


In [8]:
history.history

{'loss': [0.34321969747543335,
  0.15656515955924988,
  0.11409363150596619,
  0.0913287028670311],
 'sparse_categorical_accuracy': [0.9017199873924255,
  0.953819990158081,
  0.9660199880599976,
  0.9723799824714661],
 'val_loss': [0.1845717430114746,
  0.1269584745168686,
  0.10788918286561966,
  0.09548232704401016],
 'val_sparse_categorical_accuracy': [0.9470999836921692,
  0.9620000123977661,
  0.9686999917030334,
  0.972599983215332]}

In [9]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print ("test loss, test acc", results)

#generate predictions (probabilities -- the output of the last layer
#on new data using predict
print ("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
79/79 [==============================] - 0s 2ms/step - loss: 0.0942 - sparse_categorical_accuracy: 0.9721
test loss, test acc [0.09421412646770477, 0.972100019454956]
Generate predictions for 3 samples
predictions shape: (3, 10)


In [10]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [11]:
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

In [12]:
def get_uncompiled_model():
  inputs = keras.Input(shape=(784,), name="digits")
  x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
  x = layers.Dense(64, activation="relu", name="dense_2")(x)
  outputs = layers.Dense(10, activation="softmax",name="predictions")(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

def get_compiled_model():
  model = get_uncompiled_model()
  model.compile(
      optimizer="rmsprop",
      loss="sparse_categorical_crossentropy",
      metrics=["sparse_categorical_accuracy"],
  )
  return model

In [13]:
def custom_mean_squared_error(y_true, y_pred):
  return tf.math.reduce_mean(tf.square(y_true - y_pred))

model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=custom_mean_squared_error)

#We need to one-hot encode the labels to use MSE
y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)


782/782 [==============================] - 2s 2ms/step - loss: 0.0159


In [14]:
class CustomMSE(keras.losses.Loss):
  def __init__(self, regularization_factor=0.1, name="custom_mse"):
    super().__init__(name=name)
    self.regularization_factor = regularization_factor

  def call(self, y_true, y_pred):
    mse=tf.math.reduce_mean(tf.square(y_true - y_pred))
    reg=tf.math.reduce_mean(tf.square(0.5 - y_pred))
    return mse + reg * self.regularization_factor

model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=CustomMSE())  

y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=2)


Epoch 1/2
782/782 [==============================] - 2s 2ms/step - loss: 0.0389
Epoch 2/2
782/782 [==============================] - 2s 2ms/step - loss: 0.0314
